In [47]:
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

c:\Users\johnk\anaconda3\envs\jovani\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\johnk\anaconda3\envs\jovani\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\johnk\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to 

In [19]:
from datasets import load_dataset, concatenate_datasets, Dataset
from datasets.features import Features, Translation

# List of available language pairs
lang_pairs = ["en-eo", "en-es", "en-fi", "en-fr", "en-hu", "en-it", "en-nl", "en-no", "en-pl", "en-pt", "en-ru", "en-sv"]

# Define a unified translation feature schema
translation_feature = Features({'translation': Translation(languages=["src", "tgt"])})

# Load datasets and reverse translations
datasets = []
for lang_pair in lang_pairs:
    print(f"Loading dataset for language pair: {lang_pair}")
    
    # Load only the 'train' split
    dataset = load_dataset("opus_books", lang_pair)["train"]
    
    # Extract language codes
    src_lang, tgt_lang = lang_pair.split('-')
    
    # Reverse translation pairs and remove 'id' column
    reversed_data = dataset.map(lambda x: {'translation': {'src': x['translation'][tgt_lang], 
                                                           'tgt': x['translation'][src_lang]}}, 
                                remove_columns=['translation', 'id'])  # Remove 'id' column
    
    datasets.append(reversed_data)

# Combine all datasets into one
combined_dataset = concatenate_datasets(datasets)

# Example of accessing a reversed entry
print(combined_dataset[0])


Loading dataset for language pair: en-eo


Map: 100%|██████████| 1562/1562 [00:00<00:00, 8519.65 examples/s]


Loading dataset for language pair: en-es


Map: 100%|██████████| 93470/93470 [00:03<00:00, 24396.55 examples/s]


Loading dataset for language pair: en-fi


...

Loading dataset for language pair: en-ru


Map: 100%|██████████| 17496/17496 [00:00<00:00, 20543.44 examples/s]


Loading dataset for language pair: en-sv


Map: 100%|██████████| 3095/3095 [00:00<00:00, 10199.80 examples/s]

{'translation': {'src': 'La aventuroj de Alicio en Mirlando', 'tgt': "ALICE'S ADVENTURES IN WONDERLAND"}}


In [24]:
import pandas as pd

# Convert Hugging Face dataset to Pandas DataFrame
df = combined_dataset.to_pandas()

# Display first few rows
print(df.head(10))


                                         translation
0  {'src': 'La aventuroj de Alicio en Mirlando', ...
1   {'src': 'LEWIS CARROLL', 'tgt': 'Lewis Carroll'}
2  {'src': 'ĈAPITRO I Mirinda Falego!', 'tgt': 'C...
3  {'src': 'Alicio, jam longan tempon sidinte apu...
4  {'src': 'Ŝi do ekpripensis—ne tre vigle ĉar la...
5  {'src': 'Tio ja ne estis tre mirinda, ankaŭ ne...
6  {'src': '(Kiam poste ŝi pripensadis la aferon,...
7  {'src': 'Ne konsiderinte eĉ unu momenton, kiam...
8  {'src': 'La tunelo, ĉe la unuaj paŝoj, konduki...
9  {'src': 'Nu, aŭ la ŝakto estis efektive tre pr...


In [35]:
df.shape

(462222, 2)

In [36]:
df.head(5)

,src,tgt
0,La aventuroj de Alicio en Mirlando,ALICE'S ADVENTURES IN WONDERLAND
1,LEWIS CARROLL,Lewis Carroll
2,ĈAPITRO I Mirinda Falego!,CHAPTER I Down the Rabbit-Hole
3,"Alicio, jam longan tempon sidinte apud sia fra...",Alice was beginning to get very tired of sitti...
4,Ŝi do ekpripensis—ne tre vigle ĉar la tago est...,So she was considering in her own mind (as wel...


In [37]:
print(df.columns)  # Check the actual column names


Index(['src', 'tgt'], dtype='object')


In [38]:
print(df.head())  # Preview the first few rows


                                                 src  \
0                 La aventuroj de Alicio en Mirlando   
1                                      LEWIS CARROLL   
2                          ĈAPITRO I Mirinda Falego!   
3  Alicio, jam longan tempon sidinte apud sia fra...   
4  Ŝi do ekpripensis—ne tre vigle ĉar la tago est...   

                                                 tgt  
0                   ALICE'S ADVENTURES IN WONDERLAND  
1                                      Lewis Carroll  
2                     CHAPTER I Down the Rabbit-Hole  
3  Alice was beginning to get very tired of sitti...  
4  So she was considering in her own mind (as wel...  


In [43]:
import pandas as pd

# Sample list of food-related keywords (expand as needed)
food_keywords = [
    "butter", "sugar", "milk", "flour", "egg", "cheese", "meat", "chicken", "fish",
    "fruit", "vegetable", "carrot", "apple", "banana", "salt", "pepper", "oil", "garlic",
    "tomato", "onion", "potato", "spinach", "lettuce", "broccoli", "beef", "pork", "lamb",
    "turkey", "sausage", "bacon", "rice", "pasta", "bread", "cereal", "yogurt", "honey", 
    "chocolate", "vanilla", "lemon", "lime", "orange", "strawberry", "blueberry", "raspberry",
    "pineapple", "watermelon", "mango", "peach", "plum", "grape", "cucumber", "zucchini",
    "cauliflower", "pumpkin", "beans", "lentils", "soy", "tofu", "almond", "walnut", "cashew",
    "hazelnut", "pistachio", "peanut", "olive", "coconut", "avocado", "ginger", "turmeric",
    "cinnamon", "nutmeg", "clove", "cardamom", "paprika", "chili", "oregano", "basil", "thyme",
    "rosemary", "parsley", "cilantro", "mint", "sage", "dill", "bay leaf", "chives", "tarragon",
    "salmon", "tuna", "cod", "eggplant", "mushrooms", "soy sauce", "vinegar", "mustard", "tomato paste",
    "coconut milk", "coconut cream", "fish sauce", "sesame oil", "sriracha", "wasabi", "miso",
    "sour cream", "cream cheese", "cream", "heavy cream", "ricotta", "mozzarella", "cheddar",
    "parmesan", "feta", "pecans", "macadamia", "pine nuts", "poppy seeds", "chia seeds", "sunflower seeds",
    "sesame seeds", "pumpkin seeds", "barley", "quinoa", "oats", "corn", "cornstarch", "molasses",
    "agave", "maple syrup", "corn syrup", "beet sugar", "raisins", "dates", "figs", "apricot", "plum",
    "sultanas", "toffee", "marzipan", "nougat", "custard", "pudding", "ice cream", "whipped cream",
    "coffee", "tea", "cocoa powder", "espresso", "balsamic vinegar", "apple cider vinegar", "rice vinegar",
    "white vinegar", "sherry vinegar", "distilled vinegar", "wine", "red wine", "white wine", "rum",
    "vodka", "tequila", "whiskey", "gin", "brandy", "beer", "lemonade", "fruit juice", "lime juice",
    "orange juice", "pomegranate juice", "cranberry juice", "water", "sparkling water", "soda", "tonic water"
]



# Load your dataset (assuming it's a Pandas DataFrame)
# df = pd.read_csv("your_dataset.csv")  # Uncomment if loading from a CSV

# Convert to lowercase and filter for food-related terms
df_filtered = df[df.apply(
    lambda row: any(word in row['src'].lower() or word in row['tgt'].lower() for word in food_keywords), axis=1
)]

# Save the filtered dataset
#df_filtered.to_csv("filtered_ingredients_translations.csv", index=False)

# Display some results
print(f"Original dataset size: {len(df)}")
print(f"Filtered dataset size: {len(df_filtered)}")
print(df_filtered.head())


Original dataset size: 462222
Filtered dataset size: 82468
                                                  src  \
3   Alicio, jam longan tempon sidinte apud sia fra...   
6   (Kiam poste ŝi pripensadis la aferon, ŝajnis a...   
11  Sed jen por ŝi granda ĉagreno; ĉar, enrigardan...   
19  (Alicio jam antaŭe lernis en la lernĉambro kel...   
30  "Mi esperas ke ĉe la temanĝo oni ne forgesos ŝ...   

                                                  tgt  
3   Alice was beginning to get very tired of sitti...  
6   Oh dear! I shall be late!' (when she thought i...  
11  She took down a jar from one of the shelves as...  
19  (for, you see, Alice had learnt several things...  
30  'I hope they'll remember her saucer of milk at...  


In [45]:
df_filtered.head()

,src,tgt
3,"Alicio, jam longan tempon sidinte apud sia fra...",Alice was beginning to get very tired of sitti...
6,"(Kiam poste ŝi pripensadis la aferon, ŝajnis a...",Oh dear! I shall be late!' (when she thought i...
11,"Sed jen por ŝi granda ĉagreno; ĉar, enrigardan...",She took down a jar from one of the shelves as...
19,(Alicio jam antaŭe lernis en la lernĉambro kel...,"(for, you see, Alice had learnt several things..."
30,"""Mi esperas ke ĉe la temanĝo oni ne forgesos ŝ...",'I hope they'll remember her saucer of milk at...


In [46]:
df_filtered.isnull().sum()

src    0
tgt    0
dtype: int64

In [48]:

def preprocess(text):
  text=''.join([char for char in text if char not in string.punctuation])
  text=text.lower()
  word_tokenize = text.split()
  stop_words = stopwords.words('english')
  clean_words = ' '.join([word for word in word_tokenize if word not in stopwords.words('english')])
  return clean_words

In [49]:

df_filtered['clean_src'] = df_filtered['src'].apply(preprocess)
df_filtered.head()

C:\Users\johnk\AppData\Local\Temp\ipykernel_3388\630557466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['clean_src'] = df_filtered['src'].apply(preprocess)


,src,tgt,clean_src
3,"Alicio, jam longan tempon sidinte apud sia fra...",Alice was beginning to get very tired of sitti...,alicio jam longan tempon sidinte apud sia frat...
6,"(Kiam poste ŝi pripensadis la aferon, ŝajnis a...",Oh dear! I shall be late!' (when she thought i...,kiam poste ŝi pripensadis la aferon ŝajnis al ...
11,"Sed jen por ŝi granda ĉagreno; ĉar, enrigardan...",She took down a jar from one of the shelves as...,sed jen por ŝi granda ĉagreno ĉar enrigardante...
19,(Alicio jam antaŭe lernis en la lernĉambro kel...,"(for, you see, Alice had learnt several things...",alicio jam antaŭe lernis en la lernĉambro kelk...
30,"""Mi esperas ke ĉe la temanĝo oni ne forgesos ŝ...",'I hope they'll remember her saucer of milk at...,mi esperas ke ĉe la temanĝo oni ne forgesos ŝi...


In [50]:
df_filtered = df_filtered.copy()

In [51]:
df_filtered['clean_tgt'] = df_filtered['tgt'].apply(preprocess)
df_filtered.head()

,src,tgt,clean_src,clean_tgt
3,"Alicio, jam longan tempon sidinte apud sia fra...",Alice was beginning to get very tired of sitti...,alicio jam longan tempon sidinte apud sia frat...,alice beginning get tired sitting sister bank ...
6,"(Kiam poste ŝi pripensadis la aferon, ŝajnis a...",Oh dear! I shall be late!' (when she thought i...,kiam poste ŝi pripensadis la aferon ŝajnis al ...,oh dear shall late thought afterwards occurred...
11,"Sed jen por ŝi granda ĉagreno; ĉar, enrigardan...",She took down a jar from one of the shelves as...,sed jen por ŝi granda ĉagreno ĉar enrigardante...,took jar one shelves passed labelled orange ma...
19,(Alicio jam antaŭe lernis en la lernĉambro kel...,"(for, you see, Alice had learnt several things...",alicio jam antaŭe lernis en la lernĉambro kelk...,see alice learnt several things sort lessons s...
30,"""Mi esperas ke ĉe la temanĝo oni ne forgesos ŝ...",'I hope they'll remember her saucer of milk at...,mi esperas ke ĉe la temanĝo oni ne forgesos ŝi...,hope theyll remember saucer milk teatime
